In [1]:
# !pip install redis
# docker run --rm --name some-redis -p 6379:6379 -d redis

In [2]:
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# read local .env file
# find_dotenv 函数通常用于搜索和定位目录树中的 .env 文件，而 load_dotenv 函数用于将 .env 文件中的变量加载到环境中。
_ = load_dotenv(find_dotenv())

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)
# embeddings = OpenAIEmbeddings()

In [3]:
from langchain_core.prompts import PromptTemplate

# rom langchain import hub
# prompt = hub.pull("hwchase17/react")
# add {my_chat_history}
# or
# prompt = hub.pull("stepbystep/conversational-agent")
prompt_template = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Previous conversation history:
{my_chat_history}

Question: {input}
Thought:{agent_scratchpad}
"""
prompt = PromptTemplate.from_template(prompt_template)
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'my_chat_history', 'tool_names', 'tools'], template='\nAnswer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nPrevious conversation history:\n{my_chat_history}\n\nQuestion: {input}\nThought:{agent_scratchpad}\n')

In [4]:
""" output
PromptTemplate(input_variables=['agent_scratchpad', 'input', 'my_chat_history', 'tool_names', 'tools'], template='\nAnswer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\n{my_chat_history}\n\nQuestion: {input}\nThought:{agent_scratchpad}\n')
"""

" output\nPromptTemplate(input_variables=['agent_scratchpad', 'input', 'my_chat_history', 'tool_names', 'tools'], template='\nAnswer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\n{my_chat_history}\n\nQuestion: {input}\nThought:{agent_scratchpad}\n')\n"

In [5]:
from langchain_core.tools import Tool
from langchain_community.utilities import SerpAPIWrapper

search = SerpAPIWrapper()

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]


In [6]:
from langchain.memory import ConversationBufferMemory
# from langchain.memory.chat_memory import ChatMessageHistory
from langchain.memory.chat_message_histories import RedisChatMessageHistory
"""
现在，我们可以创建Memory对象，创建RedisChatMessageHistory以连接到外部数据库中存储的消息。
"""
message_history = RedisChatMessageHistory(url='redis://localhost:6379/0', ttl=600, session_id='my-session')

memory = ConversationBufferMemory(memory_key="my_chat_history", chat_memory=message_history)


In [7]:
from langchain.agents import create_react_agent, AgentExecutor


agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, memory=memory, verbose=True)

In [9]:
agent_executor.invoke({"input": "How many people live in canada?"})




> Entering new AgentExecutor chain...
I should search for the current population of Canada.
Action: Search
Action Input: "current population of Canada"{'type': 'population_result', 'place': 'Canada', 'population': '38.25 million', 'year': '2021'}I now know the final answer.
Final Answer: The current population of Canada is 38.25 million as of 2021.

> Finished chain.


{'input': 'How many people live in canada?',
 'my_chat_history': '',
 'output': 'The current population of Canada is 38.25 million as of 2021.'}

In [ ]:
""" output


[1m> Entering new AgentExecutor chain...[0m
[32;1m[1;3mI should search for the current population of Canada.
Action: Search
Action Input: "current population of Canada"[0m[36;1m[1;3m{'type': 'population_result', 'place': 'Canada', 'population': '38.25 million', 'year': '2021'}[0m[32;1m[1;3mI now know the final answer.
Final Answer: The current population of Canada is 38.25 million as of 2021.[0m

[1m> Finished chain.[0m

{'input': 'How many people live in canada?',
 'my_chat_history': '',
 'output': 'The current population of Canada is 38.25 million as of 2021.'}
"""

In [10]:
memory.chat_memory.messages

[HumanMessage(content='How many people live in canada?'),
 AIMessage(content='The current population of Canada is 38.25 million as of 2021.')]

In [ ]:
""" output
[HumanMessage(content='How many people live in canada?'),
 AIMessage(content='The current population of Canada is 38.25 million as of 2021.')]
"""

In [11]:
"""
要测试此代理的记忆力，我们可以提出一个后续问题，该问题依赖于先前交换中的信息才能正确回答。
"""
agent_executor.invoke({"input": "what is their national anthem called?"})



> Entering new AgentExecutor chain...
I should search for the name of Canada's national anthem.
Action: Search
Action Input: "Canada national anthem"['"O Canada" is the national anthem of Canada. The song was originally commissioned by Lieutenant Governor of Quebec Théodore Robitaille for the 1880 Saint-Jean-Baptiste Day ceremony; Calixa Lavallée composed the music, after which French-language words were written by the poet and judge Sir Adolphe-Basile Routhier.', 'O Canada type: National anthem.', 'O Canada lyricists: Calixa Lavallée, Adolphe-Basile Routhier, Robert Stanley Weir.', 'O Canada composer: Calixa Lavallée.', 'O Canada key: G major.', 'O Canada adopted: July 1, 1980.', 'O Canada also_known_as: French - Ô Canada.', 'O Canada, national anthem of Canada. It was proclaimed the official national anthem on July 1, 1980. “God Save the Queen” remains the royal ...', '"O Canada" (French: Ô Canada) is the national anthem of Canada. The song was originally commissioned by Lieutenant

{'input': 'what is their national anthem called?',
 'my_chat_history': 'Human: How many people live in canada?\nAI: The current population of Canada is 38.25 million as of 2021.',
 'output': 'The national anthem of Canada is called "O Canada".'}

In [ ]:
""" output


[1m> Entering new AgentExecutor chain...[0m
[32;1m[1;3mI should search for the name of Canada's national anthem.
Action: Search
Action Input: "Canada national anthem"[0m[36;1m[1;3m['"O Canada" is the national anthem of Canada. The song was originally commissioned by Lieutenant Governor of Quebec Théodore Robitaille for the 1880 Saint-Jean-Baptiste Day ceremony; Calixa Lavallée composed the music, after which French-language words were written by the poet and judge Sir Adolphe-Basile Routhier.', 'O Canada type: National anthem.', 'O Canada lyricists: Calixa Lavallée, Adolphe-Basile Routhier, Robert Stanley Weir.', 'O Canada composer: Calixa Lavallée.', 'O Canada key: G major.', 'O Canada adopted: July 1, 1980.', 'O Canada also_known_as: French - Ô Canada.', 'O Canada, national anthem of Canada. It was proclaimed the official national anthem on July 1, 1980. “God Save the Queen” remains the royal ...', '"O Canada" (French: Ô Canada) is the national anthem of Canada. The song was originally commissioned by Lieutenant Governor of Quebec Théodore Robitaille ...', "“O Canada” was proclaimed Canada's national anthem in 1980, a century after it was first sung in 1880. The music was composed by Calixa Lavallée ...", '"O Canada" was proclaimed Canada\'s national anthem on July 1,. 1980, 100 years after it was first sung on June 24, 1880. The music.', 'God keep our land glorious and free! O Canada, we stand on guard for thee. O Canada, we stand on guard for thee. O Canada.', 'O Canada! Our home and native land! True patriot love in all of us command. With glowing hearts we see thee rise, The True North strong and free ...', 'Canadian National Anthem - Lyrics. 3.1.2 a. O Canada! O Canada! Our home and native land! True patriot love in all thy sons command. With glowing hearts we ...', '"A Place to Stand, a Place to Grow (Ontari-ari-ari-o!)" is an unofficial anthem of Ontario. The song was written by Dolores Claman, with English lyrics by ...'][0m[32;1m[1;3mI now know the final answer
Final Answer: The national anthem of Canada is called "O Canada".[0m

[1m> Finished chain.[0m

{'input': 'what is their national anthem called?',
 'my_chat_history': 'Human: How many people live in canada?\nAI: The current population of Canada is 38.25 million as of 2021.',
 'output': 'The national anthem of Canada is called "O Canada".'}
"""

In [12]:
agent_executor.memory.buffer

'Human: How many people live in canada?\nAI: The current population of Canada is 38.25 million as of 2021.\nHuman: what is their national anthem called?\nAI: The national anthem of Canada is called "O Canada".'

In [ ]:
""" output
'Human: How many people live in canada?\nAI: The current population of Canada is 38.25 million as of 2021.\nHuman: what is their national anthem called?\nAI: The national anthem of Canada is called "O Canada".'
"""